In [ ]:

import pandas as pd
import os
from scipy.interpolate import interp1d
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from matplotlib import pyplot as plt
import networkx as nx

In [ ]:
path = r'datasets\\datasets_data\\collected\\'


In [ ]:
#数据读取与预处理
xlsx = []
csv = []

#ignored: ecodata, air_condition

def read(path,date_key='时间',date_format = None,operations=None,dropsubset=None):
    sheet = pd.read_csv(path,index_col= date_key,parse_dates=[date_key],date_format=date_format)
    sheet.rename(columns={date_key:'date'},inplace=True)
    sheet.dropna(subset=dropsubset)
    if operations is not None:
        sheet = operations(sheet)
    sheet = sheet.resample('YE').mean()
    return sheet

# xlsx.append[([pd.read_excel(os.path.join(path,'carbon_emission.xlsx'))],'emission vector')]

csv.append(read(os.path.join(path,'air_pollution_events.csv'),operations=lambda x: x.filter(regex='大气污染事故次数:年')))
csv.append(read(os.path.join(path,'citylize.csv')))
csv.append(read(os.path.join(path,'comsume.csv'),'时间?',operations=lambda x : x.filter(regex='支出')))
csv.append(read(os.path.join(path,'controllable_income.csv'),'时间'))
csv.append(read(os.path.join(path,'energy_investment.csv'),operations= lambda x: x.filter(regex='固定资产投资额(不含农户):能源工业:上海市:年')))
csv.append(read(os.path.join(path,'energy_structure.csv'),'时间?',operations=lambda x: x.filter(['能源生产总量构成:水电、核电、风电:年?','一次能源生产量:年?'])))
# csv.append(read(os.path.join(path,'medic_insurance.csv'),operations=lambda sheet: sheet.resample('YE').mean())) # 数据太少，影响整体质量，不使用
csv.append(read(os.path.join(path,'thick_cause_death.csv'),'时间?',operations= lambda x: x.filter(regex='(农村呼吸系统疾病粗死亡率_)|(城市呼吸系统疾病粗死亡率_)')))
csv.append(read(os.path.join(path,'work_time.csv'),operations=lambda sheet: sheet.resample('YE').mean()))
csv.append(read(r'D:\huge_program_project\TJJM-Time-LLM\datasets\datasets_data\collected\processed\gdp\上海市_gdp_yearly_sum.csv',date_key='年份',operations=lambda x: x.rename(columns={'总量（亿元）':'gdp'})))
csv.append(read(r'D:\huge_program_project\TJJM-Time-LLM\datasets\datasets_data\collected\processed\co2\上海_emission.csv',date_key='year',operations=lambda x:x.rename(columns={'emission':'co2_emission'})))
csv.append(read(r'D:\huge_program_project\TJJM-Time-LLM\datasets\datasets_data\collected\processed\aqi\上海市_yearly_aircondition.csv',date_key='年份',operations=lambda x:x.rename(columns={'AQI':'当年平均空气质量指数'})))


csvs = pd.concat(csv,axis=1)

# regex = '.*Road$'
# mask = csvs.apply(
#     lambda col: col.str.contains(regex, regex=True, na=False)
# ).any(axis=1) & csvs.notnull().all(axis=1)

# csvs = csvs.loc[mask]
rem = csvs.columns.difference(['date'])

for col in rem:
    csvs[col] = csvs[col].interpolate(method='spline', order=1)
    csvs[col] = csvs[col].dropna()
    
# 找到全为非空值的行
mask = csvs.notnull().all(axis=1)

# 将布尔掩码转换为int
mask = mask.astype(int)

# 计算差异和累积和以生成组ID
group_ids = mask.diff().ne(0).cumsum()

# 找到最长的组
longest_group = group_ids.groupby(group_ids).size().idxmax()

# 切片原始DataFrame以获取该组
result = csvs.loc[group_ids != longest_group]
result.index.name= 'date'

result.rename(columns={
    'date': '日期', 
    '大气污染事故次数:年': '大气污染事故次数',
    '乡村人口数:年': '乡村人口数',
    '城镇人口数:年': '城镇人口数',
    '城乡居民比例': '城乡人口比',
    '农村家庭平均每人年消费性支出:年?': '农村人均消费支出',
    '城镇居民人均消费性支出:年?': '城镇人均消费支出',
    '居民可支配收入': '可支配收入',
    '能源生产总量构成:水电、核电、风电:年?': '清洁能源占比',
    '一次能源生产量:年?': '一次能源生产量', 
    '城市呼吸系统疾病粗死亡率_当期值_年?': '城市呼吸系统疾病死亡率',
    '农村呼吸系统疾病粗死亡率_当期值_年?': '农村呼吸系统疾病死亡率',
    '就业人员周平均工作时间:当期值:月': '周均工作时间',
    'gdp': 'GDP',
    'co2_emission': 'CO2排放量',
    '当年平均空气质量指数': '空气质量指数'
}, inplace=True)

#神秘代码，但就是能用

result.to_csv(os.path.join(path,'processed','summarize.csv'))

In [ ]:
#数据标准化
import joblib
# 读取summarize.csv
df = pd.read_csv(r'datasets\datasets_data\collected\processed\summarize.csv',index_col='date',parse_dates=['date'])
# 将'date'列转换为datetime类型
# df['date'] = pd.to_datetime(df['date'])

# 检查数据质量
print("数据形状:", df.shape)
# print("数据类型:\n", df.dtypes)
# print("缺失值检查:\n", df.isnull().sum())

# 假设整理后的DataFrame变量名为df
# 选择需要标准化的列
columns = ['大气污染事故次数', '乡村人口数', '城镇人口数', '农村人均消费支出', 
                    '城镇人均消费支出', '可支配收入', '清洁能源占比', '一次能源生产量',
                    '城市呼吸系统疾病死亡率', '农村呼吸系统疾病死亡率', '周均工作时间', 
                    'GDP', 'CO2排放量', '空气质量指数']

# 初始化StandardScaler
scaler = StandardScaler().fit(df[columns])

# 对选定的列进行标准化
df[columns] = scaler.transform(df[columns])

# print("预处理后的数据:\n", df.head())
df.to_csv(os.path.join(path,'processed',"data_preprocessed.csv"))  # 保存预处理后的数据
joblib.dump(scaler, os.path.join(path,'processed',"scalar.pkl")) #保存缩放因子方便后续训练

In [ ]:
#数据分析
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = ['SimHei']  # 使用中文字体

# 读取预处理后的数据
df = pd.read_csv(os.path.join(path,'processed',"data_preprocessed.csv"), index_col='date', parse_dates=['date'])

# Granger因果检验
maxlag = 2  # 设置最大时滞
test_results = pd.DataFrame(np.zeros((len(columns), len(columns))),
                            index=columns, columns=columns)
for cause in columns:
    for effect in columns:
        if cause != effect:
            test = grangercausalitytests(list(df[[cause, effect]].dropna().values), maxlag=maxlag, verbose=False)
            p_values = [test[i+1][0]['ssr_ftest'][1] for i in range(maxlag)]
            min_p_value = np.min(p_values)
            test_results.loc[cause, effect] = min_p_value

# 交叉相关分析
lags = range(1, 11)  # 设置时滞范围
cross_corr = pd.DataFrame(index=columns, columns=columns)
for cause in columns:
    for effect in columns:
        if cause != effect:
            cross_corr.loc[cause, effect] = df[cause].shift(1).corr(df[effect])


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 20))
sns.heatmap(test_results, ax=ax1, cmap='YlGnBu', annot=False,fmt='.3f')
ax1.set_title('Granger Causality')
sns.heatmap(cross_corr.astype(float), ax=ax2, cmap='YlOrRd', annot=False,fmt='.3f')
ax2.set_title('Cross Correlations')

plt.tight_layout()
plt.show()

In [46]:
import pandas as pd
import numpy as np
import semopy
from semopy import Model

# 读取数据
# data = pd.read_csv('data_preprocessed.csv', index_col='date', parse_dates=True)
data = pd.read_csv(os.path.join(path, 'processed', 'data_preprocessed.csv'), index_col='date', parse_dates=True)


# # 添加潜变量
# model.add_lv("Health")
# model.add_lv("Environment")
# model.add_lv("Economy")
# model.add_lv("Policy")

# # 添加观测变量
# model.add_ov("大气污染事故次数", latent="Environment")
# model.add_ov("乡村人口数", latent="Economy") 
# model.add_ov("城镇人口数", latent="Economy")
# model.add_ov("城乡人口比", latent="Economy")
# model.add_ov("农村人均消费支出", latent="Economy")
# model.add_ov("城镇人均消费支出", latent="Economy") 
# model.add_ov("可支配收入", latent="Economy")
# model.add_ov("清洁能源占比", latent="Environment")
# model.add_ov("一次能源生产量", latent="Environment")
# model.add_ov("城市呼吸系统疾病死亡率", latent="Health")
# model.add_ov("农村呼吸系统疾病死亡率", latent="Health")
# model.add_ov("周均工作时间", latent="Economy")
# model.add_ov("GDP", latent="Economy") 
# model.add_ov("CO2排放量", latent="Environment")
# model.add_ov("空气质量指数", latent="Environment")

# # 添加路径
# model.add_path("Environment", "Health")  
# model.add_path("Economy", "Health")
# model.add_path("Policy", "Environment")
# model.add_path("Policy", "Economy")
# model.add_path("Economy", "Environment")

model = """
     # 潜变量
      Health =~ 城市呼吸系统疾病死亡率 + 农村呼吸系统疾病死亡率
      Environment =~ 大气污染事故次数 + 清洁能源占比 + 一次能源生产量 + CO2排放量 + 空气质量指数
      Economy =~ 乡村人口数 + 城镇人口数 + 城乡人口比 + 农村人均消费支出 + 城镇人均消费支出 + 可支配收入 + 周均工作时间 + GDP
      Policy =~ 1*Policy
    
      # 观测变量
      大气污染事故次数 ~~ NA*大气污染事故次数
      乡村人口数 ~~ NA*乡村人口数
      城镇人口数 ~~ NA*城镇人口数
      城乡人口比 ~~ NA*城乡人口比
      农村人均消费支出 ~~ NA*农村人均消费支出
      城镇人均消费支出 ~~ NA*城镇人均消费支出
      可支配收入 ~~ NA*可支配收入
      清洁能源占比 ~~ NA*清洁能源占比
      一次能源生产量 ~~ NA*一次能源生产量
      城市呼吸系统疾病死亡率 ~~ NA*城市呼吸系统疾病死亡率
      农村呼吸系统疾病死亡率 ~~ NA*农村呼吸系统疾病死亡率
      周均工作时间 ~~ NA*周均工作时间
      GDP ~~ NA*GDP
      CO2排放量 ~~ NA*CO2排放量
      空气质量指数 ~~ NA*空气质量指数
    
      # 路径关系
      Health ~ Environment + Economy
      Environment ~ Policy + Economy
      Economy ~ Policy
"""

# 定义模型
model = Model(model)

# 拟合模型
model.fit(data)

# 输出结果
# print(model.inspect())

print(semopy.calc_stats(model).T)
g = semopy.semplot(model, os.path.join("article","figures","sem_model.png"))

# 计算成本效益
def cost_benefit(policy_cost, health_benefit, discount_rate=0.05, time_horizon=20):
    net_benefit = 0
    for t in range(time_horizon):
        net_benefit += (health_benefit - policy_cost) / (1 + discount_rate)**t
    return net_benefit

policy_cost = 1000  # 假设的政策成本
health_benefit = model.predict(data, "Health", policy=1) - model.predict(data, "Health", policy=0)
health_benefit = health_benefit.mean() * 10000  # 假设每个单位对应1万元的健康效益

net_benefit = cost_benefit(policy_cost, health_benefit)
print(f"净效益: {net_benefit:.2f}")


                     Value
DoF              98.000000
DoF Baseline    119.000000
chi2                   inf
chi2 p-value      0.000000
chi2 Baseline  1114.995893
CFI                   -inf
GFI                   -inf
AGFI                  -inf
NFI                   -inf
TLI                   -inf
RMSEA                  inf
AIC                   -inf
BIC                   -inf
LogLik                 inf


LinAlgError: Singular matrix

In [ ]:
#TODO optimize
import os
import pandas as pd
import semopy
import matplotlib
matplotlib.rcParams['font.family'] = ['SimHei']  # 使用中文字体

# 读取预处理后的数据
data = pd.read_csv(os.path.join(path, 'processed', 'data_preprocessed.csv'), index_col='date', parse_dates=['date'])

# 定义潜变量和观测变量
latent_vars = ['个人层面', '微观层面', '中观层面', '宏观层面']

observed_vars = {
    '个人层面': ['个人收入', '个人健康'],
    '微观层面': ['环境污染', '能源利用'],
    '中观层面': ['环保投资', '技术水平'],
    '宏观层面': ['经济发展', '人口结构']
}

# 创建标准化数据
scaled_data = pd.DataFrame(index=data.index)
for lv, ovs in observed_vars.items():
    for ov in ovs:
        scaled_data[ov] = (data[ov] - data[ov].mean()) / data[ov].std()

# 定义lavaan模型语法
model_syntax = """
    # 潜变量定义
    个人层面 =~ 个人收入 + 个人健康
    微观层面 =~ 环境污染 + 能源利用  
    中观层面 =~ 环保投资 + 技术水平
    宏观层面 =~ 经济发展 + 人口结构
    
    # 潜变量关系
    微观层面 ~ 个人层面
    中观层面 ~ 微观层面
    宏观层面 ~ 中观层面
"""

# 创建模型
model = semopy.Model(model_syntax)

# 拟合模型参数
fitted_model = model.fit(scaled_data,obj="DWLS")

print(semopy.calc_stats(model).T)
g = semopy.semplot(model, os.path.join("article","figures","sem_model.png"))
